In [7]:
#Outbound Process

import logging
import paramiko
import sys
import shutil
import time
import posixpath
import datetime
import os
import smtplib, getpass, json

#Working with config file
with open("config.json") as f:
    config = json.load(f)

#sftp credentials
sftp_host = config["sftp"]["hostname"]
sftp_user = config["sftp"]["username"]
sftp_pass = config["sftp"]["password"]

#email credentials
email_address = config["email"]["address"]
email_pass = config["email"]["password"]


#Logging
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")
logging.basicConfig(level=logging.ERROR, filename=f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/OutboundError/{get_date}-errorlog.log", filemode="w", format="%(asctime)s - %(levelname)s - %(message)s")



#path variables
local_path = os.path.join(os.getcwd(), 'Processing/Outbound/Reprocess')
remote = '/Files/OPUS/'



with open(f"Processing/Log/Outbound/{get_date}_outbound_log", 'a') as f:
    f.write(f"============= {get_date} Outbound Processing =============\n")
    
    try:
        #Working with email
        smtp_object = smtplib.SMTP("smtp.gmail.com", 587)
        smtp_object.ehlo()
        smtp_object.starttls()
        smtp_object.ehlo()
        smtp_object.login(email_address,email_pass)
        from_address = email_address
        to_address = email_address
        subject = "Outbound Reporting"
        messages = []
    except Exception as e:
        f.write("Failed to send email:", e)

    try:
        #Paramiko instance
        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname = sftp_host, username = sftp_user, password = sftp_pass)
        f.write("Connection successful\n")
        messages.append("SFTP Connection Successful")

        try:
            with client.open_sftp() as sftp:
                remote_location = sftp.listdir(remote) #os.listdir()
                for file in remote_location:
                    if file.endswith(".pdf"):
                        source = posixpath.join(remote, file)
                        download_path = os.path.join(local_path, file)
                        sftp.get(source, download_path)
                        f.write(f"{os.path.basename(source)} downloaded to {os.path.basename(local_path)}\n") #writing to file
                        messages.append(f"{os.path.basename(source)} downloaded to {os.path.basename(local_path)}\n") #appending to messages
                        
                        sftp.remove(source) #Delete
                        f.write(f"{os.path.basename(source)} deleted\n")
                        messages.append(f"{os.path.basename(source)} deleted\n")
        except Exception as e:
            logging.exception(f"Error handling {file}: {e}")
            messages.append(f"Error handling {file}")
            sys.exit() #to stop the script if an error occurs
    except Exception as e:
        logging.exception(f"Error during SFTP connection: {e}")
        messages.append(f"Error during SFTP connection")
        sys.exit() #to stop the script if an error occurs
    finally:
        client.close()
        msg_body = "\n".join(messages)
        msg = "Subject: "+subject+'\n'+ msg_body
        smtp_object.sendmail(from_address,to_address,msg)
        smtp_object.quit()